## Context / Intro

### Failure By Model Year

In [ ]:
VTR2016 <- read.csv("cleaned_data_path")

In [ ]:
#creates new dataframes, one for cars that passed the obd test and for cars that failed the test.
#a ones column was added to help count the number of cars that passed and failed the test overall
Failed_VTR2016 <- VTR2016[which(VTR2016$V_OBD_RES == "F"),]
Failed_VTR2016 <- transform(Failed_VTR2016, ones = 1)
Passed_VTR2016 <- VTR2016[which(VTR2016$V_OBD_RES == "P"),]
Passed_VTR2016 <- transform(Passed_VTR2016, ones = 1)

In [ ]:
#aggregates the above data by model year to show how many cars of that year passed and failed the test
fails_by_model_year <- aggregate(ones~V_VEH_YEAR, Failed_VTR2016, sum)
passes_by_model_year <- aggregate(ones~V_VEH_YEAR, Passed_VTR2016, sum)
#exlcludes data past 2011, because vehicles less than 5 years old do not get tested for emissions
passes_by_model_year <- passes_by_model_year[which(passes_by_model_year$V_VEH_YEAR<=2011),]

In [ ]:
#a new table is created below in which a fail rate of the obd test is created by model year, by using a ratio of passed and failed tests from above.
new_table <- fails_by_model_year
new_table <- transform(new_table, pass_numbers = passes_by_model_year$ones)
new_table <- transform(new_table, fail_rate = 100*new_table$ones/(new_table$ones+new_table$pass_numbers) )
new_table <- transform(new_table, count_of_cars = new_table$ones+new_table$pass_numbers)

In [ ]:
failure_percent_by_model_year_2016 <- new_table[,c("V_VEH_YEAR", "fail_rate", "count_of_cars")]

### Model Year Pollution Degradation

In [ ]:
#install.packages('ggplot2')
library("ggplot2")

#Set working directory as needed
setwd('D:/R/Model_Year_Tracking_Data')

#load data set
#data includes model year of vehicle, and average pollution for HC, NOX, CO for each model year during the given testing year
#Note that years before 2010 were not included because not all testing years required vehicles with model year in the late 2000s to test
#model years from pre-2000 also did not have substantial quantities of records for all testing years and were thus not included
#model years past 2011 were not included as vehicles past this cutoff were not required to be tested yet during the time of this analysis
Model_Year_Data = read.csv("./Model_Year_Pollution_Tracking.csv",header = TRUE)



In [ ]:
#below 3 plots are created, one each for CO, HC, and NOX emissions
#plots are made so data is by plotted by vehicle year, and grouped/colored by the emissions testing year
ggplot(Model_Year_Data,aes(ï..Vehicle_Year, CO_Emissions_grams_per_mile,color=Testing_Year, group=Testing_Year)) + 
  geom_line() + 
  xlab('Vehicle Year') +
  ylab('CO Emissions (grams/mile)')+
  ggtitle('CO Emissions by Model Year and Testing Year')+
  scale_x_continuous(breaks=c(1982,1990,2000,2010))+
  scale_y_continuous(breaks=seq(0,50,10))


ggplot(Model_Year_Data,aes(ï..Vehicle_Year, HC_Emissions_grams_per_mile,color=Testing_Year, group=Testing_Year)) + 
  geom_line() + 
  xlab('Vehicle Year') +
  ylab('HC Emissions (grams/mile)')+
  ggtitle('HC Emissions by Model Year and Testing Year')+
  scale_x_continuous(breaks=c(1982,1990,2000,2010))+
  scale_y_continuous(breaks=seq(0,3.5,.5))

ggplot(Model_Year_Data,aes(ï..Vehicle_Year, NOX_Emissions_grams_per_mile,color=Testing_Year, group=Testing_Year)) + 
  geom_line() + 
  xlab('Vehicle Year') +
  ylab('NOX Emissions (grams/mile)')+
  ggtitle('NOX Emissions by Model Year and Testing Year')+
  scale_x_continuous(breaks=c(1982,1990,2000,2010))+
  scale_y_continuous(breaks=seq(0,3,.5))
  #scale_colour_manual(name = "", 
   #                   values = c("green3", "orange", "blue", "red", "grey", "purple", "yellow"))


###################################################################



In [ ]:
#now we will create a table and charts to track how the average emission value for a model year fleet changed over time
#first we initialize empty vectors for each pollutant
CO_change_vector = vector()
HC_change_vector = vector()
NOX_change_vector = vector()

#the for loop below will run through each model year, and subtract the average pollutant value for all three pollutants of the 2016 test year
#by the values in the 2010 test year
#the end results will be saved in the three vectors above
for (x in 1982:2011) {
HC_Change_value = Model_Year_Data[Model_Year_Data$Testing_Year==2016 & Model_Year_Data$ï..Vehicle_Year==x,]$HC_Emissions_grams_per_mile - Model_Year_Data[Model_Year_Data$Testing_Year==2010 & Model_Year_Data$ï..Vehicle_Year==x,]$HC_Emissions_grams_per_mile
NOX_Change_value = Model_Year_Data[Model_Year_Data$Testing_Year==2016 & Model_Year_Data$ï..Vehicle_Year==x,]$NOX_Emissions_grams_per_mile - Model_Year_Data[Model_Year_Data$Testing_Year==2010 & Model_Year_Data$ï..Vehicle_Year==x,]$NOX_Emissions_grams_per_mile
CO_Change_value = Model_Year_Data[Model_Year_Data$Testing_Year==2016 & Model_Year_Data$ï..Vehicle_Year==x,]$CO_Emissions_grams_per_mile - Model_Year_Data[Model_Year_Data$Testing_Year==2010 & Model_Year_Data$ï..Vehicle_Year==x,]$CO_Emissions_grams_per_mile

HC_change_vector = c(HC_change_vector,HC_Change_value)
NOX_change_vector = c(NOX_change_vector,NOX_Change_value)
CO_change_vector = c(CO_change_vector,CO_Change_value)

}

#change the vectors to dataframes, and create a years column to represent each model year
HC_change_vector=as.data.frame(HC_change_vector)
NOX_change_vector=as.data.frame(NOX_change_vector)
CO_change_vector=as.data.frame(CO_change_vector)
years = as.data.frame(c(1982:2011))



In [ ]:
#create new table to show model year emissions change calculations from above
Pollution_Change_df = as.data.frame(cbind(years,CO_change_vector, HC_change_vector,NOX_change_vector))

#below the emissions change data calculated above is plotted, model year vs emissions change
#note because of scale, CO is graphed separately than NOX and HC.

ggplot() + 
  geom_line(data=Pollution_Change_df,aes(x=`c(1982:2011)`, y=NOX_change_vector), color='red')+
  geom_line(data=Pollution_Change_df,aes(x=`c(1982:2011)`, y=HC_change_vector),color='blue')+
  #geom_line(data=Pollution_Change_df,aes(x=Pollution_Change_df$`c(1982:2011)`, y=Pollution_Change_df$CO_change_vector),color='green')+

  xlab('Vehicle Year') +
  ylab('Change in Emissions, 2010 to 2016, g/mile')+
  ggtitle('NOX&HC Emissions Change by Model Year, 2010 to 2016')+
  scale_x_continuous(breaks=c(1982,1990,2000,2010))
  

ggplot() + 
  geom_line(data=Pollution_Change_df,aes(x=`c(1982:2011)`, y=CO_change_vector),color='green')+
  
  xlab('Vehicle Year') +
  ylab('Change in Emissions, 2010 to 2016, g/mile')+
  ggtitle('CO Emissions Change by Model Year, 2010 to 2016')+
  scale_x_continuous(breaks=c(1982,1990,2000,2010))

#looking at the charts, we see that the value for each graph started relatively high and then decreased


In [ ]:
#################################################################################################

#the below calculations mirror those of the section above, but showing percent change rather than g/mile raw change
CO_percent_change_vector = vector()
HC_percent_change_vector = vector()
NOX_percent_change_vector = vector()

for (x in 1982:2011) {
  HC_pct_Change_value = 100*(Model_Year_Data[Model_Year_Data$Testing_Year==2016 & Model_Year_Data$ï..Vehicle_Year==x,]$HC_Emissions_grams_per_mile - Model_Year_Data[Model_Year_Data$Testing_Year==2010 & Model_Year_Data$ï..Vehicle_Year==x,]$HC_Emissions_grams_per_mile)/Model_Year_Data[Model_Year_Data$Testing_Year==2010 & Model_Year_Data$ï..Vehicle_Year==x,]$HC_Emissions_grams_per_mile
  NOX_pct_Change_value = 100*(Model_Year_Data[Model_Year_Data$Testing_Year==2016 & Model_Year_Data$ï..Vehicle_Year==x,]$NOX_Emissions_grams_per_mile - Model_Year_Data[Model_Year_Data$Testing_Year==2010 & Model_Year_Data$ï..Vehicle_Year==x,]$NOX_Emissions_grams_per_mile)/Model_Year_Data[Model_Year_Data$Testing_Year==2010 & Model_Year_Data$ï..Vehicle_Year==x,]$NOX_Emissions_grams_per_mile
  CO_pct_Change_value = 100*(Model_Year_Data[Model_Year_Data$Testing_Year==2016 & Model_Year_Data$ï..Vehicle_Year==x,]$CO_Emissions_grams_per_mile - Model_Year_Data[Model_Year_Data$Testing_Year==2010 & Model_Year_Data$ï..Vehicle_Year==x,]$CO_Emissions_grams_per_mile)/Model_Year_Data[Model_Year_Data$Testing_Year==2010 & Model_Year_Data$ï..Vehicle_Year==x,]$CO_Emissions_grams_per_mile
  
  HC_percent_change_vector = c(HC_percent_change_vector,HC_pct_Change_value)
  NOX_percent_change_vector = c(NOX_percent_change_vector,NOX_pct_Change_value)
  CO_percent_change_vector = c(CO_percent_change_vector,CO_pct_Change_value)
  
}


Pollution_Percent_Change_df = cbind(years,CO_percent_change_vector, HC_percent_change_vector,NOX_percent_change_vector)

ggplot() + 
  geom_line(data=Pollution_Percent_Change_df,aes(x=`c(1982:2011)`, y=NOX_percent_change_vector), color='red')+
  geom_line(data=Pollution_Percent_Change_df,aes(x=`c(1982:2011)`, y=HC_percent_change_vector),color='blue')+
  xlab('Vehicle Year') +
  ylab('Percent Change in Emissions, 2010 to 2016')+
  ggtitle('NOX&HC Emissions Percent Change by Model Year, 2010 to 2016')+
  scale_x_continuous(breaks=c(1982,1990,2000,2010))
  

ggplot() + 
  geom_line(data=Pollution_Percent_Change_df,aes(x=`c(1982:2011)`, y=CO_percent_change_vector),color='green')+
  
  xlab('Vehicle Year') +
  ylab('Percent Change in Emissions, 2010 to 2016, g/mile')+
  ggtitle('CO Emissions Percent Change by Model Year, 2010 to 2016')+
  scale_x_continuous(breaks=c(1982,1990,2000,2010))

#looking at the charts, we see a highly fluctuating line
#this is likely because newer model year vehicles emitted far less than older vehicles
#thus minor changes/degradation to the pollution controls of newer vehicles would still lead to high % change values
#because of this sensitivity, using the charts from section above are likely to be more useful for presentation

### Pollution Control Degradation

In [ ]:
#plot(x=seq(1982,2017),y=aggregate(V_HC~V_VEH_YEAR, VTR2010, mean)[,2]/max(aggregate(V_HC~V_VEH_YEAR, VTR2010, mean)[,2]), xlab="Vehicle Year", ylab="HC/NOX/COavg (grams/mile)", main="HC/NOX/CO Emission by Vehicle Year", pch=15, cex.lab=1.4,col="blue")
#points(x=seq(1982,2017),aggregate(V_NOX~V_VEH_YEAR,VTR2010,mean)[,2]/max(aggregate(V_NOX~V_VEH_YEAR,VTR2010,mean)[,2]),pch=15, col="red")
#points(x=seq(1982,2017),aggregate(V_CO~V_VEH_YEAR,VTR2010,mean)[,2]/max(aggregate(V_CO~V_VEH_YEAR,VTR2010,mean)[,2]),pch=15, col="green")
#legend(2008,.7, c("HC","NOX","CO"),lty=c(1,1,1),lwd=c(2.5,2.5,2.5),col=c("blue","red","green"))

#the above lines of code create plots of emissions per model year for HC, NOX, CO. They are 
#normalized to 1 so they could be plotted on the same graph

VTR2016 <- read.csv("C:/Users/Allan/Downloads/VTR2016.txt")
#loads the relevant data



In [ ]:
HC_Amount_first=aggregate(V_HC~V_VEH_YEAR, first_tests, mean)
NOX_amount_first=aggregate(V_NOX~V_VEH_YEAR,first_tests,mean)
CO_amount_first=aggregate(V_CO~V_VEH_YEAR,first_tests,mean)
#shows average emissions of first test by vehicle model year
Vehicle_year_first <- cbind(HC_Amount_first, NOX_amount_first, CO_amount_first)
write.csv(Vehicle_year_first, "C:/Users/Allan/Documents/Carnegie Mellon/Junior Year/Spring/EPP Projects/degradation/2010/Vehicle_Year_first_test_emissions_2010.csv")
#exports files as a csv file



In [ ]:
HC_Amount_Make_first=aggregate(V_HC~V_MAKE, first_tests, mean)
NOX_amount_Make_first=aggregate(V_NOX~V_MAKE,first_tests,mean)
CO_amount_Make_first=aggregate(V_CO~V_MAKE,first_tests,mean)
#shows average emissions of first test by vehicle make
Vehicle_make_first <- cbind(HC_Amount_Make_first, NOX_amount_Make_first, CO_amount_Make_first)
write.csv(Vehicle_make_first, "C:/Users/Allan/Documents/Carnegie Mellon/Junior Year/Spring/EPP Projects/degradation/2010/Vehicle_Make_first_test_emissions_2010.csv")
#exports files as a csv file

HC_Amount_Model_first=aggregate(V_HC~V_MODEL, first_tests, mean)
NOX_amount_Model_first=aggregate(V_NOX~V_MODEL,first_tests,mean)
CO_amount_Model_first=aggregate(V_CO~V_MODEL,first_tests,mean)
#shows average emissions of first test by model year
Vehicle_model_first <- cbind(HC_Amount_Model_first, NOX_amount_Model_first, CO_amount_Model_first)
write.csv(Vehicle_model_first, "C:/Users/Allan/Documents/Carnegie Mellon/Junior Year/Spring/EPP Projects/degradation/2010/Vehicle_Model_first_test_emissions_2010.csv")
#exports files as csv file

HC_Amount_VIN_first=aggregate(V_HC~new_vin, first_tests, mean)
NOX_amount_VIN_first=aggregate(V_NOX~new_vin,first_tests,mean)
CO_amount_VIN_first=aggregate(V_CO~new_vin,first_tests,mean)
#shows average emissions of first test by first 8 digits of the VIN
Vehicle_VIN_first <- cbind(HC_Amount_VIN_first, NOX_amount_VIN_first, CO_amount_VIN_first)
write.csv(Vehicle_VIN_first, "C:/Users/Allan/Documents/Carnegie Mellon/Junior Year/Spring/EPP Projects/degradation/2010/Vehicle_VIN_first_test_emissions_2010.csv")
#exports files as csv file



In [ ]:
HC_Amount_last=aggregate(V_HC~V_VEH_YEAR, last_tests, mean)
NOX_amount_last=aggregate(V_NOX~V_VEH_YEAR,last_tests,mean)
CO_amount_last=aggregate(V_CO~V_VEH_YEAR,last_tests,mean)
#shows average emissions of last test by vehicle model year
Vehicle_year_last <- cbind(HC_Amount_last, NOX_amount_last, CO_amount_last)
write.csv(Vehicle_year_last, "C:/Users/Allan/Documents/Carnegie Mellon/Junior Year/Spring/EPP Projects/degradation/2010/Vehicle_year_last_test_emissions_2010.csv")
#exports files as csv file

HC_Amount_Make_last=aggregate(V_HC~V_MAKE, last_tests, mean)
NOX_amount_Make_last=aggregate(V_NOX~V_MAKE,last_tests,mean)
CO_amount_Make_last=aggregate(V_CO~V_MAKE,last_tests,mean)
#shows average emissions of last test by vehicle make
Vehicle_Make_last <- cbind(HC_Amount_Make_last, NOX_amount_Make_last, CO_amount_Make_last)
write.csv(Vehicle_Make_last, "C:/Users/Allan/Documents/Carnegie Mellon/Junior Year/Spring/EPP Projects/degradation/2010/Vehicle_Make_last_test_emissions_2010.csv")
#exports files as csv file

HC_Amount_Model_last=aggregate(V_HC~V_MODEL, last_tests, mean)
NOX_amount_Model_last=aggregate(V_NOX~V_MODEL,last_tests,mean)
CO_amount_Model_last=aggregate(V_CO~V_MODEL,last_tests,mean)
#shows average emissions of last test by model year
Vehicle_Model_last <- cbind(HC_Amount_Model_last, NOX_amount_Model_last, CO_amount_Model_last)
write.csv(Vehicle_Model_last, "C:/Users/Allan/Documents/Carnegie Mellon/Junior Year/Spring/EPP Projects/degradation/2010/Vehicle_Model_last_test_emissions_2010.csv")
#exports files as a csv file

HC_Amount_VIN_last=aggregate(V_HC~new_vin, last_tests, mean)
NOX_amount_VIN_last=aggregate(V_NOX~new_vin,last_tests,mean)
CO_amount_VIN_last=aggregate(V_CO~new_vin,last_tests,mean)
#shows average emissions of last test by first 8 digits of the VIN
VIN_last <- cbind(HC_Amount_VIN_last, NOX_amount_VIN_last, CO_amount_VIN_last)
write.csv(VIN_last, "C:/Users/Allan/Documents/Carnegie Mellon/Junior Year/Spring/EPP Projects/degradation/2010/Vin_last_2010.csv")
#exports files as csv file


### Pollutants Saved

In [ ]:
#creates a new dataframe that includes only records for cars that eventually passed the emissions test
#for these cars, the first and last (if applicable) emissions quantities found during testing were added as columns
first_tests_new <- transform(first_tests, new_CO = last_tests$CO_adjusted)
first_tests_new <- transform(first_tests_new, new_NOX = last_tests$NOX_adjusted)
first_tests_new <- transform(first_tests_new, new_HC = last_tests$HC_adjusted)
first_tests_new <- transform(first_tests_new, last_CO = last_tests$V_CO_RES)
first_tests_new <- transform(first_tests_new, last_HC = last_tests$V_HC_RES)
first_tests_new <- transform(first_tests_new, last_NOX = last_tests$V_NOX_RES)
first_tests_new <- transform(first_tests_new, last_time = last_tests$V_DATE_TIME)



In [ ]:

#standardized time, taking it out of the original dataframe's format so timestamps can be subtracted
first_tests_new$V_DATE_TIME <- as.POSIXct(first_tests_new$V_DATE_TIME, tz = "GMT")
first_tests_new$last_time <- as.POSIXct(first_tests_new$last_time, tz = "GMT")



In [ ]:
#new field created, to measure the delta in time between the initial emissions test and the final emissions test for a given vehicle
first_tests_new <- transform(first_tests_new, delta_time = first_tests_new$last_time - first_tests_new$V_DATE_TIME)
first_tests_new <- transform(first_tests_new, new_delta = as.numeric(first_tests_new$delta_time))


In [ ]:
#filters the dataset to only include cars with failing tests
first_tests_new <- first_tests_new[first_tests_new$V_EM_RES =="F",]
first_tests_new <- first_tests_new[first_tests_new$new_delta > 1,]



In [ ]:
first_tests_new <- transform(first_tests_new, mileage_per_year = first_tests_new$V_ODOMETER/(2014 - first_tests_new$V_VEH_YEAR))
#creates new data set that holds both initial and final emissions results. Also creates a mileage per year field



In [ ]:
#creates a new field, showing the difference between initial emissions and final emissions in a vehicle. this difference is plotted
first_tests_new <- transform(first_tests_new, plotted_first_difference_CO = first_tests_new$CO_adjusted - first_tests_new$V_CO_STD)
first_tests_new <- transform(first_tests_new, plotted_last_difference_CO = first_tests_new$new_CO - first_tests_new$V_CO_STD)
plot(first_tests_new$plotted_first_difference_CO, first_tests_new$plotted_last_difference_CO)




In [ ]:
#the same process done above is repeated for all emissions tested for the vehicles, and plotted
#first_tests_CO <- first_tests_new[first_tests_new$CO_adjusted - first_tests_new$new_CO > 0 & first_tests_new$last_CO == "P",]
first_tests_CO <- first_tests_CO[first_tests_CO$V_CO_RES == "F",]
first_tests_NOX <- first_tests_new[first_tests_new$NOX_adjusted - first_tests_new$new_NOX > 0 & first_tests_new$last_NOX == "P",]
first_tests_HC <- first_tests_new[first_tests_new$HC_adjusted - first_tests_new$new_HC > 0 & first_tests_new$last_HC == "P",]
#creates new data sets to hold cars that failed a particular pollutant test


In [ ]:

first_tests_CO <- transform(first_tests_CO, CO_saved = first_tests_CO$mileage_per_year*(first_tests_CO$CO_adjusted - first_tests_CO$new_CO)/907185 )
first_tests_NOX <- transform(first_tests_NOX, NOX_saved = first_tests_NOX$mileage_per_year*(first_tests_NOX$NOX_adjusted - first_tests_NOX$new_NOX)/907185 )
first_tests_HC <- transform(first_tests_HC, HC_saved = first_tests_HC$mileage_per_year*(first_tests_HC$HC_adjusted - first_tests_HC$new_HC)/907185 )
#finds how much emissions were saved with each vehicle



In [ ]:
#emissions saved due to testing is calculated below
#this is done as follows:
#multiply the average milage travelled for the vehicle by the delta in emissions found above.
CO_saved_by_mini_vin <- aggregate(CO_saved~mini_vin, first_tests_CO, sum)
CO_saved_by_mini_vin <- CO_saved_by_mini_vin[with(CO_saved_by_mini_vin, order(CO_saved, decreasing = TRUE)),]
#finds the CO saved by miniVIN

Nox_saved_by_mini_vin <- aggregate(NOX_saved~mini_vin, first_tests_NOX, sum)
Nox_saved_by_mini_vin <- Nox_saved_by_mini_vin[with(Nox_saved_by_mini_vin, order(NOX_saved, decreasing = TRUE)),]
#finds the NOX saved by minivin

HC_saved_by_mini_vin <- aggregate(HC_saved~mini_vin, first_tests_HC, sum)
HC_saved_by_mini_vin <- HC_saved_by_mini_vin[with(HC_saved_by_mini_vin, order(HC_saved, decreasing = TRUE)),]
#finds the HC saved by miniVIN



In [ ]:
#the above values, initially per mini_vin, are aggregated to find total emissions saved due to emissions testing below.
total_CO_saved <- sum(CO_saved_by_mini_vin$CO_saved)
total_NOX_saved <- sum(Nox_saved_by_mini_vin$NOX_saved)
total_HC_saved <- sum(HC_saved_by_mini_vin$HC_saved)
#finds the total emissions saved




In [ ]:
#pollutants lost shown below.

#same procedure done as above, but for vehicles that failed their last emissions test
#this helps identify potential emissions lost due to cars that continue driving for a period after failing inspection

first_tests_CO_failed_last <- first_tests_new[first_tests_new$CO_adjusted - first_tests_new$new_CO < 0 & first_tests_new$last_CO == "F",]
first_tests_NOX_failed_last <- first_tests_new[first_tests_new$NOX_adjusted - first_tests_new$new_NOX < 0 & first_tests_new$last_CO == "F",]
first_tests_HC_failed_last <- first_tests_new[first_tests_new$HC_adjusted - first_tests_new$new_HC < 0 & first_tests_new$last_CO == "F",]
#creates new data sets to hold cars that failed a particular pollutant test

first_tests_CO_failed_last <- transform(first_tests_CO_failed_last, CO_lost = first_tests_CO_failed_last$mileage_per_year*(first_tests_CO_failed_last$CO_adjusted - first_tests_CO_failed_last$new_CO)/907185 )
first_tests_NOX_failed_last <- transform(first_tests_NOX_failed_last, NOX_lost = first_tests_NOX_failed_last$mileage_per_year*(first_tests_NOX_failed_last$NOX_adjusted - first_tests_NOX_failed_last$new_NOX)/907185 )
first_tests_HC_failed_last <- transform(first_tests_HC_failed_last, HC_lost = first_tests_HC_failed_last$mileage_per_year*(first_tests_HC_failed_last$HC_adjusted - first_tests_HC_failed_last$new_HC)/907185 )
#finds how much emissions were saved with each vehicle

CO_lost_by_mini_vin <- aggregate(CO_lost~mini_vin, first_tests_CO_failed_last, sum)
CO_lost_by_mini_vin <- CO_lost_by_mini_vin[with(CO_lost_by_mini_vin, order(CO_lost, decreasing = TRUE)),]
#finds the CO saved by miniVIN


HC_lost_by_mini_vin <- aggregate(HC_lost~mini_vin, first_tests_HC_failed_last, sum)
HC_lost_by_mini_vin <- HC_lost_by_mini_vin[with(HC_lost_by_mini_vin, order(HC_lost, decreasing = TRUE)),]
#finds the CO saved by miniVIN

NOX_lost_by_mini_vin <- aggregate(NOX_lost~mini_vin, first_tests_NOX_failed_last, sum)
NOX_lost_by_mini_vin <- NOX_lost_by_mini_vin[with(NOX_lost_by_mini_vin, order(NOX_lost, decreasing = TRUE)),]
#finds the CO saved by miniVIN

total_CO_lost <- sum(CO_lost_by_mini_vin$CO_lost)
total_NOX_lost <- sum(NOX_lost_by_mini_vin$NOX_lost)
total_HC_lost <- sum(HC_lost_by_mini_vin$HC_lost)

